In [1]:
#import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools

In [ ]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image

In [ ]:
real_image_path = '/kaggle/input/casia-dataset/casia/CASIA2/Au/Au_ani_00001.jpg'
Image.open(real_image_path)

In [ ]:
convert_to_ela_image(real_image_path, 90)

In [ ]:
fake_image_path = '/kaggle/input/casia-dataset/casia/CASIA2/Tp/Tp_D_NRN_S_N_ani10171_ani00001_12458.jpg'
Image.open(fake_image_path)

In [ ]:
convert_to_ela_image(fake_image_path, 90)

In [ ]:
# Prepare image function
image_size = (128, 128)
def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 90).resize(image_size)).flatten() / 255.0

In [ ]:

# Initialize X and Y as empty lists
X = []  # List to hold the images
Y = []  # List to hold the labels (0 for fake, 1 for real)

# Process the 'real' images (label = 1)
path = '/kaggle/input/casia-dataset/CASIA2/Au/'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png'):
            full_path = os.path.join(dirname, filename)
            X.append(prepare_image(full_path))  # Append to the list
            Y.append(1)  # Label as real
            if len(Y) % 500 == 0:
                print(f'Processing {len(Y)} images')

# Process the 'fake' images (label = 0)
path = '/kaggle/input/casia-dataset/CASIA2/Tp/'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png'):
            full_path = os.path.join(dirname, filename)
            X.append(prepare_image(full_path))  # Append to the list
            Y.append(0)  # Label as fake
            if len(Y) % 500 == 0:
                print(f'Processing {len(Y)} images')

# Convert lists to numpy arrays
X = np.array(X)
Y = np.array(Y)

print(f"Length of X: {len(X)}, Length of Y: {len(Y)}")


In [ ]:
# Convert labels to categorical format
Y = to_categorical(Y, 2)

# Reshape X for CNN input
X = X.reshape(-1, 128, 128, 3)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)
X = X.reshape(-1, 128, 128, 3)
print(len(X_train), len(Y_train))
print(len(X_val), len(Y_val))

In [ ]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (5, 5), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
    model.add(Conv2D(filters = 32, kernel_size = (5, 5), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))
    return model

In [ ]:
model = build_model()
model.summary()
Model: "sequential"

In [ ]:
epochs = 30
batch_size = 32
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

# Initialize the learning rate and optimizer
init_lr = 1e-4
optimizer = Adam(learning_rate=init_lr)

# Define the learning rate schedule function
def lr_schedule(epoch, lr):
    decay_rate = init_lr / epochs
    return init_lr * (1 / (1 + decay_rate * epoch))

# Create the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

# Compile the model with the optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, verbose=1)

# Fit the model with early stopping
hist = model.fit(X_train,
                 Y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(X_val, Y_val),
                 callbacks=[early_stopping])

In [ ]:
model.save('forgery_abd_model.h5')